In [6]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download&id={}"
    session = requests.Session()
    response = session.get(URL.format(id), stream=True)
    token = get_confirm_token(session, response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL.format(id), params = params, stream = True)
    save_response_content(response, destination)

def get_confirm_token(session, response):
    for key, value in response.cookies.items():
        if key.startswith('download'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

download_file_from_google_drive('1_CQ7RdIkje47n1q4HigrQdgplNwQVTlO', 'data.csv')

In [6]:
import pandas as pd

df = pd.read_csv('data.csv')
display(df.head())

,For Haryana Delhi & Chandigarh starting the check from FEB in the latest year (2017):
0,Predicted consecutive drought months (4):
1,2017-FEB: 11.50 mm
2,2017-MAR: 7.20 mm
3,2017-APR: 2.80 mm
4,2017-MAY: 8.00 mm


In [1]:
import pandas as pd

hydroponics_df = pd.read_csv('/content/hydrophonics.csv')
display(hydroponics_df.head())

,species,type,min_water_mL_per_plant_per_day,recommended_water_mL_per_plant_per_day,notes
0,Lettuce (Lactuca sativa),Leafy green,50,150,Seedlings use much less; mature heads need ste...
1,Spinach (Spinacia oleracea),Leafy green,50,200,Similar to lettuce but slightly higher once ma...
2,Kale (Brassica oleracea var. sabellica),Leafy green,75,250,Hardier; tolerates wider fluctuations.
3,Basil (Ocimum basilicum),Herb,50,200,Prefers consistent moisture and higher EC cont...
4,Tomato (Solanum lycopersicum),Fruiting crop,200,1200,Varies a lot with variety and fruiting stage; ...


In [2]:
display(hydroponics_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   species                                 10 non-null     object
 1   type                                    10 non-null     object
 2   min_water_mL_per_plant_per_day          10 non-null     int64 
 3   recommended_water_mL_per_plant_per_day  10 non-null     int64 
 4   notes                                   10 non-null     object
dtypes: int64(2), object(3)
memory usage: 532.0+ bytes


None

In [3]:
hydroponics_df_sorted_by_min_water = hydroponics_df.sort_values(by='min_water_mL_per_plant_per_day')
display(hydroponics_df_sorted_by_min_water)

,species,type,min_water_mL_per_plant_per_day,recommended_water_mL_per_plant_per_day,notes
8,Microgreens (mixed),Microgreen,5,20,Very small volumes; often hand-misted or shall...
9,"Herb (mixed — parsley, cilantro)",Herb,40,180,General herb guidance; adjust by species.
0,Lettuce (Lactuca sativa),Leafy green,50,150,Seedlings use much less; mature heads need ste...
1,Spinach (Spinacia oleracea),Leafy green,50,200,Similar to lettuce but slightly higher once ma...
3,Basil (Ocimum basilicum),Herb,50,200,Prefers consistent moisture and higher EC cont...
2,Kale (Brassica oleracea var. sabellica),Leafy green,75,250,Hardier; tolerates wider fluctuations.
7,Strawberry (Fragaria × ananassa),Fruiting crop,100,400,"Depends on cultivar and system (gutter, NFT, o..."
6,Bell pepper (Capsicum annuum),Fruiting crop,150,800,Moderate water compared to tomato/cucumber.
4,Tomato (Solanum lycopersicum),Fruiting crop,200,1200,Varies a lot with variety and fruiting stage; ...
5,Cucumber (Cucumis sativus),Fruiting crop,300,1500,High transpiration during fruiting; trellised ...


In [9]:
species_name = input("Enter the species name: ")
num_plants = int(input("Enter the number of plants: "))
num_drought_months = int(input("Enter the number of drought months: "))
# Find the minimum water requirement for the specified species
species_row = hydroponics_df[hydroponics_df['species'].str.contains(species_name, case=False)]

if not species_row.empty:
    min_water_per_plant_per_day = species_row['min_water_mL_per_plant_per_day'].iloc[0]

    # Calculate the total required water
    total_required_water_mL = num_plants * min_water_per_plant_per_day * 30 * num_drought_months

    print(f"The total required water for {num_plants} {species_name} plants over {num_drought_months} drought months is: {total_required_water_mL:.2f} mL")
    print(f"This is equal to {total_required_water_mL / 1000000:.2f} cubic meters.")

else:
    print(f"Species '{species_name}' not found in the dataset.")

Enter the species name: Strawberry
Enter the number of plants: 20
Enter the number of drought months: 4
The total required water for 20 Strawberry plants over 4 drought months is: 240000.00 mL
This is equal to 0.24 cubic meters.


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
output_text = f"The total required water for {num_plants} {species_name} plants over {num_drought_months} drought months is: {total_required_water_mL:.2f} mL\nThis is equal to {total_required_water_mL / 1000000:.2f} cubic meters."

file_path = '/content/drive/MyDrive/water_requirement_output.txt'

with open(file_path, 'w') as f:
    f.write(output_text)

print(f"Output saved to {file_path}")

Output saved to /content/drive/MyDrive/water_requirement_output.txt
